In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook


symptoms = ['tos', 'dysphonia', 'tachypnea', 'ausc_resp', 'odynophagia', 'nasal_congestion', 'fatiga', 'headache']
covid_pvals = [x/10 for x in range(1,9)]           #Exemple
cold_pvals = [(x-5)**2/20 for x in range(1,9)]     #Exemple
"""
...
other diseases' pvals
...
"""

x = np.arange(len(symptoms))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, covid_pvals, width, label='Covid', color='skyblue')
rects2 = ax.bar(x + width/2, cold_pvals, width, label='Cold', color='salmon')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('P-value')
ax.set_ylim(0, 1)
ax.set_title('P-values for each disease')
ax.set_xticks(x)
ax.set_xticklabels(symptoms)
ax.legend()

for tick in ax.get_xticklabels():
    tick.set_rotation(55)

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)


plt.rcdefaults()
fig, ax = plt.subplots()

# Example data
variables = ['tos', 'dysphonia', 'tachypnea', 'ausc_resp', 'odynophagia', 'nasal_congestion', 'fatiga', 'headache']
pvals = [(x-5)**2/20 for x in range(1,9)]   # example

dict1 = {variables[i] : pvals[i] for i in range(len(variables))}
sorted_keys = sorted(dict1, key=dict1.get, reverse=True)  # [1, 3, 2]
dict2 = {w : dict1[w] for w in sorted_keys}

y_pos = np.arange(len(variables))
pvals = [dict2[var] for var in dict2.keys()]

ax.barh(y_pos, pvals, 0.4, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(sorted_keys,fontsize=5,wrap=False)

ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('P-value')
ax.set_title('Influence on covid')

plt.show()